In [ ]:
!pip install pade
import pickle
import random
from pade.misc.utility import display_message, start_loop
from pade.core.agent import Agent
from pade.acl.aid import AID
from pade.acl.messages import ACLMessage
from enum import Enum

In [ ]:
MAGIC_SQUARE = [[8, 3, 4], [1, 5, 9], [6, 7, 2]]


class Player(Enum):
    HUMAN = 1
    COMPUTER = 2
    NONE = 3


PLAYER_X = "X"
PLAYER_O = "O"


class TicTacToe:
    def __init__(self):
        self.n = 3
        self.board = [
            list(range(start, start + self.n))
            for start in list(range(1, self.n * self.n + 1, self.n))
        ]
        self.current_player = PLAYER_X
        self.player_1_list = []
        self.player_2_list = []
        self.player_1 = Player.NONE
        self.player_2 = Player.NONE
        self.outcome = ""
        self.winner = Player.NONE

    def __is_computer_first_player(self):
        return bool(random.getrandbits(1))

    def random_select_first_player(self):
        if self.__is_computer_first_player():
            self.player_1 = Player.COMPUTER
            self.player_2 = Player.HUMAN
            self.mark_square(5)
            print("Computer goes first it marks square: 5")
        else:
            self.player_1 = Player.HUMAN
            self.player_2 = Player.COMPUTER

    def print_board(self):
        for r in self.board:
            print(end="| ")
            for val in r:
                print(val, end=" | ")
            print()

    def check_square_marked(self, val):
        if val == PLAYER_O or val == PLAYER_X:
            return True
        return not any(val in x for x in self.board)

    def empty_squares_list(self):
        result = []
        for r in range(self.n):
            for c in range(self.n):
                if self.board[r][c] != PLAYER_O and self.board[r][c] != PLAYER_X:
                    result.append(self.board[r][c])
        return result

    def mark_square(self, pos):
        for r in range(self.n):
            for c in range(self.n):
                sqr_value = self.board[r][c]
                if sqr_value == pos:
                    self.board[r][c] = self.current_player
                    if self.current_player == PLAYER_X:
                        self.player_1_list.append(MAGIC_SQUARE[r][c])
                        self.current_player = PLAYER_O
                    elif self.current_player == PLAYER_O:
                        self.player_2_list.append(MAGIC_SQUARE[r][c])
                        self.current_player = PLAYER_X
                    return

    def unmark_square(self, pos):
        self.winner = Player.NONE
        self.outcome = ""
        sqr_number = 1
        if self.current_player == PLAYER_X:
            self.player_2_list.pop()
            self.current_player = PLAYER_O
        else:
            self.player_1_list.pop()
            self.current_player = PLAYER_X
        for r in range(self.n):
            for c in range(self.n):
                if sqr_number == pos:
                    self.board[r][c] = sqr_number
                    return
                sqr_number += 1

  

    def _has_won(self, player_list):
        player_list_len = len(player_list)
        for i in range(player_list_len):
            for j in range(i + 1, player_list_len):
                for k in range(j + 1, player_list_len):
                    if player_list[i] + player_list[j] + player_list[k] == 15:
                        return True
        return False

    def check_winner(self):
        if self._has_won(self.player_1_list):
            if self.player_1 == Player.COMPUTER:
                self.outcome = "COMPUTER WON!!"
                self.winner = Player.COMPUTER
            else:
                self.outcome = "HUMAN WON!!"
                self.winner = Player.HUMAN
            return True
        if self._has_won(self.player_2_list):
            if self.player_2 == Player.COMPUTER:
                self.outcome = "COMPUTER WON!!"
                self.winner = Player.COMPUTER
            else:
                self.outcome = "HUMAN WON!!"
                self.winner = Player.HUMAN
            return True
        return False

    def get_winner(self):
        return self.winner

    def check_draw(self):
        for row in self.board:
            for val in row:
                if not (val == "X" or val == "O"):
                    return False
        self.outcome = "DRAW!!"
        return True

    def check_game_over(self):
        return self.check_winner() or self.check_draw() or False

  
  
class HumanAgent(Agent):
    def __init__(self, aid, computer_agent, tic_tac_toe):
        super().__init__(aid)
        self.computer_agent = computer_agent
        self.tic_tac_toe = tic_tac_toe

    def on_start(self):
        super().on_start()
        display_message(self.aid.localname, "Human Turn")
        self.tic_tac_toe.print_board()
        sqr_number = int(input("Enter square number: "))
        while self.tic_tac_toe.check_square_marked(sqr_number):
            print("Square already marked. Try again!")
            sqr_number = int(input("Enter square number: "))
        self.tic_tac_toe.mark_square(sqr_number)
        self.send_message()

    def react(self, message):
        super().react(message)
        self.tic_tac_toe = pickle.loads(message.content)
        if self.tic_tac_toe.check_game_over():
            print()
            print(self.tic_tac_toe.outcome)
            return
        display_message(self.aid.localname, "human turn")
        sqr_number = int(input("enter square number: "))
        while self.tic_tac_toe.check_square_marked(sqr_number):
            print("square already marked. try again!")
            sqr_number = int(input("enter square number: "))
        self.tic_tac_toe.mark_square(sqr_number)
        self.tic_tac_toe.print_board()
        self.send_message()

    def send_message(self):
        message = ACLMessage(ACLMessage.INFORM)
        message.set_protocol(ACLMessage.FIPA_REQUEST_PROTOCOL)
        message.add_receiver(self.computer_agent)
        self.add_all_agents(message.receivers)
        tic_tac_toe = pickle.dumps(self.tic_tac_toe)
        message.set_content(tic_tac_toe)
        self.send(message)

    def add_all_agents(self, receivers):
        for receiver in receivers:
            self.agentInstance.table[receiver.localname] = receiver


class ComputerAgent(Agent):
    def __init__(self, aid, tic_tac_toe):
        super().__init__(aid)
        self.tic_tac_toe = tic_tac_toe

    def react(self, message):
        super().react(message)
        self.tic_tac_toe = pickle.loads(message.content)
        if self.tic_tac_toe.check_game_over():
            print()
            print(self.tic_tac_toe.outcome)
            return
        display_message(self.aid.localname, "Computer's turn")
        sqr_number = self.make_decision()
        self.tic_tac_toe.mark_square(sqr_number)
        print("Computer marks cell: {}".format(sqr_number))
        self.tic_tac_toe.print_board()
        self.send_message(message.sender.name)

    def send_message(self, receiver_agent):
        message = ACLMessage(ACLMessage.INFORM)
        message.set_protocol(ACLMessage.FIPA_REQUEST_PROTOCOL)
        message.add_receiver(receiver_agent)
        self.add_all_agents(message.receivers)
        tic_tac_toe = pickle.dumps(self.tic_tac_toe)
        message.set_content(tic_tac_toe)
        self.send(message)

    def add_all_agents(self, receivers):
        for receiver in receivers:
            self.agentInstance.table[receiver.localname] = receiver

    def make_decision(self):
        best_sqr = 0
        best_score = float("-inf")
        a=float("-inf")
        b=float("inf")
        empty_squares_list = self.tic_tac_toe.empty_squares_list()
        for sqr_value in empty_squares_list:
            if not self.tic_tac_toe.check_square_marked(sqr_value):
                self.tic_tac_toe.mark_square(sqr_value)
                score = self.minimax(0,a,b,False)
                self.tic_tac_toe.unmark_square(sqr_value)
                if score > best_score:
                    best_sqr = sqr_value
                    best_score = score
        return best_sqr

    def get_score(self) -> float:
        winner_player = self.tic_tac_toe.get_winner()
        if winner_player == Player.COMPUTER:
            return 10
        if winner_player == Player.HUMAN:
            return -10
        return 0

    def minimax(self,depth,alpha,beta,is_maximizing):
        if self.tic_tac_toe.check_game_over():
            return self.get_score()
        if is_maximizing:
            best_score = float("-inf")
            empty_squares_list = self.tic_tac_toe.empty_squares_list()
            for sqr_value in empty_squares_list:
                if not self.tic_tac_toe.check_square_marked(sqr_value):
                    self.tic_tac_toe.mark_square(sqr_value)
                    score = self.minimax(depth+1,alpha,beta,False)
                    self.tic_tac_toe.unmark_square(sqr_value)
                    best_score = max(score, best_score)
                    alpha=max(alpha,score)
                    if beta <= alpha:
                      break
            return best_score
        else:
            best_score = float("inf")
            empty_squares_list = self.tic_tac_toe.empty_squares_list()
            for sqr_value in empty_squares_list:
                if not self.tic_tac_toe.check_square_marked(sqr_value):
                    self.tic_tac_toe.mark_square(sqr_value)
                    score = self.minimax(depth+1,alpha,beta,True)
                    self.tic_tac_toe.unmark_square(sqr_value)
                    best_score = min(score, best_score)
                    beta=min(beta,score)
                    if beta <= alpha:
                      break
            return best_score


if __name__ == "__main__":
    tic_tac_toe = TicTacToe()
    tic_tac_toe.random_select_first_player()
    agents = list()
    human_agent_aid = AID(name="human_agent@localhost:{}".format(30005))
    computer_agent_aid = AID(name="computer_agent@localhost:{}".format(30006))
    computer_agent = ComputerAgent(computer_agent_aid, tic_tac_toe)
    human_agent = HumanAgent(human_agent_aid, computer_agent_aid, tic_tac_toe)
    agents.append(human_agent)
    agents.append(computer_agent)
    start_loop(agents)

Computer goes first it marks square: 5
[human_agent] 19/02/2021 05:39:45.247 --> Human Turn
| 1 | 2 | 3 | 
| 4 | X | 6 | 
| 7 | 8 | 9 | 
Enter square number: 7
[computer_agent] 19/02/2021 05:39:49.717 --> Computer's turn
Computer marks cell: 1
| X | 2 | 3 | 
| 4 | X | 6 | 
| O | 8 | 9 | 
[human_agent] 19/02/2021 05:39:49.751 --> human turn
enter square number: 3
| X | 2 | O | 
| 4 | X | 6 | 
| O | 8 | 9 | 
[computer_agent] 19/02/2021 05:40:00.659 --> Computer's turn
Computer marks cell: 2
| X | X | O | 
| 4 | X | 6 | 
| O | 8 | 9 | 
[human_agent] 19/02/2021 05:40:00.671 --> human turn
enter square number: 4
| X | X | O | 
| O | X | 6 | 
| O | 8 | 9 | 
[computer_agent] 19/02/2021 05:40:15.440 --> Computer's turn
Computer marks cell: 6
| X | X | O | 
| O | X | X | 
| O | 8 | 9 | 
[human_agent] 19/02/2021 05:40:15.447 --> human turn
enter square number: 8
| X | X | O | 
| O | X | X | 
| O | O | 9 | 
[computer_agent] 19/02/2021 05:40:33.456 --> Computer's turn
Computer marks cell: 9
| X | 